In [45]:
from nba_api.stats.static import teams
import pandas as pd
import requests
import sqlconfig
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
#Identifying all NBA teams
def nba_teams():
    try:
        nbaTeams=teams._get_teams()
        print("Fetched Team Data")
    except requests.exceptions.Timeout:
        print("Time out Error")
        nbaTeams=[]

    #Data Cleaning Dictonary to dataframe    
    nbaTeams=pd.json_normalize(nbaTeams)
    
    #Extracting columns for BCNF

    #NBA TEAM STATES
    print("NBA Team States")
    NBA_team_states=nbaTeams[['id','state']]
    NBA_team_states.columns=['team_id','team_state']
    print(NBA_team_states.head(5))
    

    #NBA team city
    print("NBA Team City")
    NBA_team_city=nbaTeams[['id','city']]
    NBA_team_city.columns=['team_id','team_city']
    print(NBA_team_city.head(5))
    

    #NBATEAMS
    print("NBATEAMS")
    teams_nba=nbaTeams[['id','full_name','abbreviation','nickname','year_founded']]
    teams_nba = teams_nba.copy()
    teams_nba['url']=teams_nba['id'].apply(lambda x: f'https://cdn.nba.com/logos/nba/{x}/primary/L/logo.svg')
    teams_nba.columns=['team_id','full_name','abbreviation','nickname','year_founded','team_url_logo']
    print(teams_nba.head(5))

    return teams_nba,NBA_team_city,NBA_team_states

In [ ]:
def intitalInsertion(data,tablename):
    try:
        engine = create_engine(
            f"postgresql+psycopg2://{sqlconfig.DB_USER}:{sqlconfig.DB_PASSWORD}@"
            f"{sqlconfig.DB_HOST}:{sqlconfig.DB_PORT}/{sqlconfig.DB_NAME}"
        )
        
        # Insertions into the tables
        data.to_sql(tablename, engine, schema="nba24_25", if_exists="append", index=False)
      
        print("Data inserted")

    except SQLAlchemyError as e:
        print(f"Database error: {e}")

    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
teams_nba,NBA_team_city,NBA_team_states=nba_teams()

intitalInsertion(teams_nba,"nbateams")
intitalInsertion(NBA_team_city,"nba_team_city")
intitalInsertion(NBA_team_states,"nba_team_states")


Fetched Team Data
NBA Team States
      team_id     team_state
0  1610612737        Georgia
1  1610612738  Massachusetts
2  1610612739           Ohio
3  1610612740      Louisiana
4  1610612741       Illinois
NBA Team City
      team_id    team_city
0  1610612737      Atlanta
1  1610612738       Boston
2  1610612739    Cleveland
3  1610612740  New Orleans
4  1610612741      Chicago
NBATEAMS
      team_id             full_name abbreviation   nickname  year_founded  \
0  1610612737         Atlanta Hawks          ATL      Hawks          1949   
1  1610612738        Boston Celtics          BOS    Celtics          1946   
2  1610612739   Cleveland Cavaliers          CLE  Cavaliers          1970   
3  1610612740  New Orleans Pelicans          NOP   Pelicans          2002   
4  1610612741         Chicago Bulls          CHI      Bulls          1966   

                                       team_url_logo  
0  https://cdn.nba.com/logos/nba/1610612737/prima...  
1  https://cdn.nba.com/logos/nba/1